<font style='font-size:1.5em'>**Initial Data Analysis**</font>

<font style='font-size:1.2em'>Using Reddit CSV</font>

**Author: Sentipedia**  

**Course: DS105L Project** 

**DATE: 3/29/2023** 

---


## Imports 

### Importing Libraries 

In [22]:
# importing libraries for web scrapping and API acess
import requests
from bs4 import BeautifulSoup

# importing libraries for dataframes 
import pandas as pd
import numpy as np

# importing libraries for plots 
from plotnine import *

# importing libraries for ploting networks
import networkx as nx
import matplotlib.pyplot as plt

# Data

### Looking at the Data 

In [5]:
# import language model 
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# downloading NLTK analyzers 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /Users/ap/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ap/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ap/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [23]:
df_reddit = pd.read_csv('reddit.csv', usecols=['comment'])
df_reddit.head()

,comment
0,"Agree. With the Tories pushing Brexit, they have become a resting place for racists as well as ..."
1,I just thought that with plenty of vaccinations coming in and a Brexit deal finally secured wed...
2,"If you don't listen to your core voter base, they won't vote for you. That's what labour did in ..."
3,"Much as I agree with your potential projections, we are not overwhelmingly Conservative (capital..."
4,"Brexit, the word, is why it succeeded. Nobody would have enthusiastically supported ""the withdra..."


In [29]:
df_reddit_full = pd.read_csv('reddit.csv')
df_reddit_full

### Using NLTK

In [28]:
df_reddit_nltk = df_reddit.head(50).copy()

analyzer = SentimentIntensityAnalyzer()
i = 0
df_reddit_nltk['pos'] = None
df_reddit_nltk['score'] = None

for comment in df_reddit_nltk['comment']:
     tokens = nltk.word_tokenize(df_reddit_nltk['comment'][i])
     df_reddit_nltk['pos'][i] = nltk.pos_tag(tokens)
     df_reddit_nltk['score'][i] = analyzer.polarity_scores(df_reddit_nltk['comment'][i])
     i = i + 1

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
df_reddit_nltk

,comment,pos,score
0,"Agree. With the Tories pushing Brexit, they have become a resting place for racists as well as right wing anti socialism.","[(Agree, NNP), (., .), (With, IN), (the, DT), (Tories, NNPS), (pushing, VBG), (Brexit, NNP), (,, ,), (they, PRP), (have, VBP), (become, VBN), (a, DT), (resting, JJ), (place, NN), (for, IN), (racists, NNS), (as, RB), (well, RB), (as, IN), (right, JJ), (wing, VBG), (anti, JJ), (socialism, NN), (., .)]","{'neg': 0.22, 'neu': 0.606, 'pos': 0.174, 'compound': -0.296}"
1,"I just thought that with plenty of vaccinations coming in and a Brexit deal finally secured wed be more hopeful and optimistic, but nah.","[(I, PRP), (just, RB), (thought, VBD), (that, IN), (with, IN), (plenty, NN), (of, IN), (vaccinations, NNS), (coming, VBG), (in, IN), (and, CC), (a, DT), (Brexit, NNP), (deal, NN), (finally, RB), (secured, VBN), (wed, NN), (be, VB), (more, RBR), (hopeful, JJ), (and, CC), (optimistic, JJ), (,, ,), (but, CC), (nah, RB), (., .)]","{'neg': 0.063, 'neu': 0.705, 'pos': 0.232, 'compound': 0.5152}"
2,"If you don't listen to your core voter base, they won't vote for you. That's what labour did in the last election. You can believe it what ever you want, but if you think the people who voted for Brexit and Tories are the ones detached from reality and not the leftists who can't make progressive policies that anyone wants to vote for then I don't know what to tell ya","[(If, IN), (you, PRP), (do, VBP), (n't, RB), (listen, VB), (to, TO), (your, PRP$), (core, NN), (voter, NN), (base, NN), (,, ,), (they, PRP), (wo, MD), (n't, RB), (vote, VB), (for, IN), (you, PRP), (., .), (That, DT), ('s, VBZ), (what, WP), (labour, NN), (did, VBD), (in, IN), (the, DT), (last, JJ), (election, NN), (., .), (You, PRP), (can, MD), (believe, VB), (it, PRP), (what, WP), (ever, RB), (you, PRP), (want, VBP), (,, ,), (but, CC), (if, IN), (you, PRP), (think, VBP), (the, DT), (people, NNS), (who, WP), (voted, VBD), (for, IN), (Brexit, NNP), (and, CC), (Tories, NNPS), (are, VBP), (the, DT), (ones, NNS), (detached, VBN), (from, IN), (reality, NN), (and, CC), (not, RB), (the, DT), (leftists, NNS), (who, WP), (ca, MD), (n't, RB), (make, VB), (progressive, JJ), (policies, NNS), (that, IN), (anyone, NN), (wants, VBZ), (to, TO), (vote, VB), (for, IN), (then, RB), (I, PRP), (do, VBP), (n't, RB), (know, VB), (what, WP), (to, TO), (tell, VB), (ya, NN)]","{'neg': 0.025, 'neu': 0.959, 'pos': 0.016, 'compound': -0.1531}"
3,"Much as I agree with your potential projections, we are not overwhelmingly Conservative (capital C), but the way our elections work exaggerated results to imply that we are. The tories got 56% of the seats with 43.6% of the votes having gained 7.5% of seats with a 1.2 percentage point swing in votes. This is with a protest vote over brexit and the relentless vilification of Corbyn by the press. It wont be long before people hit back and vote against the tories but if Scotland do ever leave the union then it will be a lot less likely.","[(Much, JJ), (as, IN), (I, PRP), (agree, VBP), (with, IN), (your, PRP$), (potential, JJ), (projections, NNS), (,, ,), (we, PRP), (are, VBP), (not, RB), (overwhelmingly, RB), (Conservative, JJ), ((, (), (capital, JJ), (C, NNP), (), )), (,, ,), (but, CC), (the, DT), (way, NN), (our, PRP$), (elections, NNS), (work, VBP), (exaggerated, JJ), (results, NNS), (to, TO), (imply, VB), (that, IN), (we, PRP), (are, VBP), (., .), (The, DT), (tories, NNS), (got, VBD), (56, CD), (%, NN), (of, IN), (the, DT), (seats, NNS), (with, IN), (43.6, CD), (%, NN), (of, IN), (the, DT), (votes, NNS), (having, VBG), (gained, VBN), (7.5, CD), (%, NN), (of, IN), (seats, NNS), (with, IN), (a, DT), (1.2, CD), (percentage, NN), (point, NN), (swing, NN), (in, IN), (votes, NNS), (., .), (This, DT), (is, VBZ), (with, IN), (a, DT), (protest, NN), (vote, NN), (over, IN), (brexit, NN), (and, CC), (the, DT), (relentless, JJ), (vilification, NN), (of, IN), (Corbyn, NNP), (by, IN), (the, DT), (press, NN), (., .), (It, PRP), (w

### Using DistilBERT Model

In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

In [7]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'LABEL_1'

In [9]:
df_reddit_DistilBERT = df_reddit.head(50).copy()
i = 0
df_reddit_DistilBERT['predicted'] = None

for comment in df_reddit_DistilBERT['comment']:
     inputs = tokenizer(df_reddit_DistilBERT['comment'][i], return_tensors="pt")
     with torch.no_grad():
          logits = model(**inputs).logits

     predicted_class_id = logits.argmax().item()
     df_reddit_DistilBERT['predicted'][i] = model.config.id2label[predicted_class_id]
     i = i + 1

df_reddit_DistilBERT

,comment,predicted
0,"Agree. With the Tories pushing Brexit, they h...",LABEL_1
1,I just thought that with plenty of vaccination...,LABEL_1
2,"If you don't listen to your core voter base, t...",LABEL_1
3,Much as I agree with your potential projection...,LABEL_1
4,"Brexit, the word, is why it succeeded. Nobody ...",LABEL_1
5,"""Brexit means Brexit""... =""\n\nThe one stand o...",LABEL_1
6,The word actually wasnt really used much anyw...,LABEL_1
7,"It was used, but only as a kind of novelty ter...",LABEL_1
8,This is exact right - the portmanteau of Grex...,LABEL_1
9,Its probably somewhere between the two of us ...,LABEL_1


In [12]:
text = "This is an example sentence to encode using DistilBERT."
encoded_text = tokenizer.encode("[CLS] " + text + " [SEP]")
input_ids = torch.tensor([encoded_text])
outputs = model(input_ids)
hidden_states = outputs[0]

In [13]:
input_ids

tensor([[  101,   101,  2023,  2003,  2019,  2742,  6251,  2000,  4372, 16044,
          2478,  4487, 16643, 23373,  1012,   102,   102]])

In [14]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1000, -0.0733]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)